In [2]:
import numpy as np
from IPython.display import HTML, display
import tabulate
import random
import datetime
import sys
from copy import deepcopy
import pickle

In [3]:
names=[
['Кленовый лист','Сакура','Сосна','Ирис'],
['Восходящее солнце','Тандзаку','Птица','Дождь']
]

In [4]:
class Card:
    def __init__(self, a,b,player):
        self.a = a
        self.b = b
        self.player=player

In [5]:
def draw_card(card):
    print(names[0][card.a]+' ['+str(card.player)+'] '+names[1][card.b])

In [6]:
def generate_field():
    cards=[]
    for i in range(0,4):
        for j in range(0,4):
            s=Card(i,j,0)
            cards.append(s)
    cards=np.array(cards)
    np.random.shuffle(cards)
    return np.reshape(cards,(4,4))

In [7]:
def draw_field(field):
    field_map=[]
    for i in field:
        s=[]
        for j in i:
            s.append(names[0][j.a]+' ['+str(j.player)+'] '+names[1][j.b])
        field_map.append(s)
    display(HTML(tabulate.tabulate(field_map, tablefmt='html',colalign=("center", "center","center", "center"))))

In [8]:
def place_available(universe,place):
    place_card=universe.field[place[0]][place[1]]
    available_places=[
                [0,0],[0,1],[0,2],[0,3],
                [1,0],[1,3],
                [2,0],[2,3],
                [3,0],[3,1],[3,2],[3,3]
            ]
    
    availability=True

    if (not universe.parent=='') and \
       (not universe.parent.card=='') and \
       (not names[0][place_card.a] in [names[0][universe.card.a],names[1][universe.card.b]]) and \
       (not names[1][place_card.b] in [names[0][universe.card.a],names[1][universe.card.b]]):
        
        availability=False
    
    elif universe.turn==0 and not place in available_places:
        
        availability=False

    elif not place_card.player==0:
        
        availability=False

    return availability

In [9]:
def draw_tree(universe):
    cnt=1
    for u in universe.child_universes:
        cnt+=draw_tree(u)
    return cnt

In [10]:
def universe_winner(universe):
    p_a=[-1,-1,-1,-1]
    p_b=[1,1,1,1]
    field=universe.field
    #horisontal
    for row in field:
        line=[c.player for c in row]        
        if line==p_a:
            return -1
        if line==p_b:            
            return 1
    #vertical
    for col in range(0,4):
        line=[]
        for row in range(0,4):
            line.append(field[row][col].player)
        if line==p_a:
            return -1
        if line==p_b:            
            return 1
    #quad
    for comp in [-1,1]:
        for row in range(0,3):
            for col in range(0,3):                
                if field[0+row][0+col].player==comp and \
                   field[0+row][1+col].player==comp and \
                   field[1+row][0+col].player==comp and \
                   field[1+row][1+col].player==comp:
                    return comp
    return 0

In [11]:
class Env:
    def __init__(self, parent,place):
        self.child_universes=[]
        self.parent=parent
        self.wins_A=0
        self.wins_B=0
        if parent=='':
            self.player=-1            
            self.turn=0
            self.field=generate_field()
            
        else:            
            self.player=-1 if parent.player==1 else 1            
            self.turn=parent.turn+1
            self.field=deepcopy(parent.field)
            self.field[place[0]][place[1]].player=self.player
        self.card=self.field[place[0]][place[1]]

In [12]:
def back_propogation(universe,winner):
    if winner==-1:
        universe.wins_A+=1
    elif winner==1:
        universe.wins_B+=1
    if universe.turn>0:
        back_propogation(universe.parent,winner)

In [13]:
def create_subuniverses(universe):
    universes_added=0
    if not universe_winner(universe):
        for i in range(0,4):
            for j in range(0,4):            
                if place_available(universe,[i,j]):                    
                    universes_added+=1
                    universe.child_universes.append( Env(universe,[i,j]) )
    if universes_added==0:
        back_propogation(universe,universe.player)
    return universes_added

In [14]:
def get_child(universe,depth,cell):
    child=universe
    for i in range(0,depth):
        child=child.child_universes[cell]
    return child

In [15]:
def draw_party(universe,depth,cell):
    for i in range(0,depth):
        u=universe if i ==0 else get_child(universe,i,cell)
        print('turn:',u.turn)
        print('player:',u.player)
        draw_card(u.card)
        draw_field(u.field)

In [16]:
def get_children(universe,turn):
    container=[]
    if universe.turn==turn-1:
        return universe.child_universes
    for child in universe.child_universes:
        container+=get_children(child,turn)
    return container

In [17]:
def play(universe_count,universes,start,depth):
    for turn in range(start,depth):
        children=get_children(universe,turn)
        print('turn:',turn,'mother universes:',len(children))
        for child in children:
            universe_count+=create_subuniverses(child)
        print(datetime.datetime.now(),'universe family:',universe_count,'wins',universe.wins_A,'x',universe.wins_B)
    print('complete')

In [38]:
def get_best_child(universe):
    children=universe.child_universes
    best_child=universe.child_universes[0]
    if best_child.player==-1:
        for child in children:
            if child.wins_A>best_child.wins_A:
                best_child=child
        print('wins:',best_child.wins_A)
    else:
        for child in children:
            if child.wins_B>best_child.wins_B:
                best_child=child
        print('wins:',best_child.wins_B)
    draw_card(best_child.card)
    return best_child

In [44]:
def get_last_universe(universe,way):
    for turn in way:
        for child in universe.child_universes:
            if child.card.a==turn.a and child.card.b==turn.b:
                universe=child
                break
    return universe

In [48]:
# 10 universe family: 5461240 wins 4484 x 48035
# 11 universe family: 14362751 wins 152165 x 38707 - 25 min
universe=Env('',[0,0])
universe_count=1
universe_count+= create_subuniverses(universe)
print(datetime.datetime.now(),'universe family:',universe_count)

play(universe_count,universe,1,11)

2020-02-25 00:15:33.995965 universe family: 13
turn: 1 mother universes: 12
2020-02-25 00:15:34.008991 universe family: 85 wins 0 x 0
turn: 2 mother universes: 72
2020-02-25 00:15:34.066010 universe family: 445 wins 0 x 0
turn: 3 mother universes: 360
2020-02-25 00:15:34.290939 universe family: 2101 wins 0 x 0
turn: 4 mother universes: 1656
2020-02-25 00:15:35.373916 universe family: 9445 wins 0 x 0
turn: 5 mother universes: 7344
2020-02-25 00:15:40.742049 universe family: 40117 wins 0 x 0
turn: 6 mother universes: 30672
2020-02-25 00:15:58.358521 universe family: 157621 wins 0 x 0
turn: 7 mother universes: 117504
2020-02-25 00:17:02.941110 universe family: 568365 wins 0 x 809
turn: 8 mother universes: 410744
2020-02-25 00:20:30.827515 universe family: 1870809 wins 3814 x 809
turn: 9 mother universes: 1302444
2020-02-25 00:30:07.753865 universe family: 5477227 wins 3814 x 45560
complete


In [75]:
pickle.dump(universe, file = open("okiya_11.pickle", "wb"))

In [ ]:
# Start game there !
#universe = pickle.load(open("okiya_3.pickle", "rb"))

In [70]:
# Every turn, add places of name list. for example, 3,2=='Сосна','Тандзаку'
#last parameter user does not matter there
way=[
    Card(3,2,1),
    Card(3,1,-1),
    Card(3,0,1),
    Card(3,3,-1),
    Card(0,3,1),
    Card(0,1,-1),
    Card(2,1,1),
    Card(2,3,-1)
]
last=get_last_universe(universe,way)
print('turn',last.turn)
draw_field(last.field)
get_best_child(last)